Tests for state, decision or reporting dependence in DFF, referencing each behavior to another single condition (such as quiet) across all voxels in the brain

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pickle
from time import time

import numpy as np
import pandas as pd

from janelia_core.utils.data_saving import append_ts
from janelia_core.stats.regression import grouped_linear_regression_ols_estimator
from janelia_core.stats.regression import grouped_linear_regression_acm_stats

from keller_zlatic_vnc.data_processing import count_unique_subjs_per_transition
from keller_zlatic_vnc.data_processing import extract_transitions
from keller_zlatic_vnc.linear_modeling import one_hot_from_table

## Parameters go here

In [3]:
ps = {}

# Specigy where the processed data (the produced of dff_extraction.ipynb) is stored
ps['data_folder'] = r'A:\projects\keller_vnc\results\whole_brain_stats'
ps['data_file'] = r'dff_1_5_5_with_ep_2020_08_27_12_32_48_526097.pkl'

# Speccify a cut-off time
ps['cut_off_time'] = 3.656

# Specify manipulation target
ps['manip_type'] = 'A4' # 'both', 'A4' or 'A9'

# Specify thresholds for number of subjects we need for each behavior
ps['min_n_subjects_per_beh'] = 3

# Specify the test type  Options are:
#
#   state_dependence - tests if dff after manipulation is sensitive to behavior before
#   prediction_dependence - tests if dff before manipulation is sensitive to behavior after
#   decision_dependence - tests if dff during manipulation is sensitive to behavior after
#   before_reporting - tests if dff before manipulation is sensitive to behavior before
#   after_reporting - tests if dff after manipulation is sensitive to behavior after
#
ps['test_type'] = 'prediction_dependence'

# Specify reference behavior
ps['beh_ref'] = 'Q'

# Alpha value for thresholding significance
ps['alpha'] = .05

# Specify where we save results
ps['save_folder'] = r'\\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats'
ps['save_str'] = 'quiet_ref_A4'

## Load the data

In [4]:
data_path = Path(ps['data_folder']) / Path(ps['data_file'])
with open(data_path, 'rb') as f:
    file_data = pickle.load(f)
data = file_data['event_annots']

## Rename a few columns

In [5]:
data.rename(columns = {'Smp ID':'subject_id', 'Beh Before':'beh_before', 'Beh After':'beh_after'}, inplace = True) 

## Apply cut-off time to define succeeding quiet behaviors

In [6]:
_, data = extract_transitions(data, ps['cut_off_time'])

## Down select for manipulation target if needed

In [7]:
if ps['manip_type'] == 'A4':
    data = data[data['Tgt Site'] == 'A4']
elif ps['manip_type'] == 'A9':
    data = data[data['Tgt Site'] == 'A9']

## Remove behaviors which are not present in enough subjects

After removing these behaviors, we keep only events which start and stop with retained behaviors

In [8]:
trans_subj_cnts = count_unique_subjs_per_transition(data)

In [9]:
if (ps['test_type'] == 'state_dependence') or (ps['test_type'] == 'before_reporting'):
    after_beh_th = 0
    before_beh_th = ps['min_n_subjects_per_beh']
elif ((ps['test_type'] == 'prediction_dependence') or (ps['test_type'] == 'after_reporting') or 
      (ps['test_type'] == 'decision_dependence')):
    after_beh_th = ps['min_n_subjects_per_beh']
    before_beh_th = 0
else:
    raise(ValueError('The test_type ' + ps['test_type'] + ' is not recognized.'))

In [10]:
after_beh_sum = trans_subj_cnts.sum()
after_behs = [b for b in after_beh_sum[after_beh_sum >= after_beh_th].index]

before_beh_sum = trans_subj_cnts.sum(1)
before_behs = [b for b in before_beh_sum[before_beh_sum >= before_beh_th].index]

before_keep_rows = data['beh_before'].apply(lambda x: x in set(before_behs))
after_keep_rows = data['beh_after'].apply(lambda x: x in set(after_behs))
data = data[before_keep_rows & after_keep_rows]

## Update our list of before and after behaviors

We do this since by removing rows, some of our control behaviors may no longer be present

In [11]:
new_trans_sub_cnts = count_unique_subjs_per_transition(data)
new_after_beh_sum = new_trans_sub_cnts.sum()
after_behs = [b for b in new_after_beh_sum[new_after_beh_sum > 0].index]
new_before_beh_sum = new_trans_sub_cnts.sum(1)
before_behs = [b for b in new_before_beh_sum[new_before_beh_sum>0].index]
print('Using the following before behaviors: ' + str(before_behs))
print('Using the following after behaviors: ' + str(after_behs))
print(['Number of rows remaining in data: ' + str(len(data))])

Using the following before behaviors: ['B', 'F', 'H', 'O', 'Q', 'T']
Using the following after behaviors: ['B', 'F', 'O', 'P', 'Q', 'T']
['Number of rows remaining in data: 154']


## Pull out $\Delta F/F$

In [12]:
if (ps['test_type'] == 'state_dependence') or (ps['test_type'] == 'after_reporting'):
    dff = np.stack(data['dff_after'].to_numpy())
    print('Extracting dff after the manipulation.')
elif (ps['test_type'] == 'prediction_dependence') or (ps['test_type'] == 'before_reporting'):
    dff = np.stack(data['dff_before'].to_numpy())
    print('Extracting dff before the manipulation.')
elif ps['test_type'] == 'decision_dependence':
    dff = np.stack(data['dff_during'].to_numpy())
    print('Extracting dff during the manipulation.')
else:
    raise(ValueError('The test_type ' + ps['test_type'] + ' is not recognized.'))

Extracting dff before the manipulation.


## Find grouping of data by subject

In [13]:
unique_ids = data['subject_id'].unique()
g = np.zeros(len(data))
for u_i, u_id in enumerate(unique_ids):
    g[data['subject_id'] == u_id] = u_i

## Define a function for calculating stats

In [14]:
def stats_f(x_i, y_i, g_i, alpha_i):
    beta, acm, n_grps = grouped_linear_regression_ols_estimator(x=x_i, y=y_i, g=g_i)
    stats = grouped_linear_regression_acm_stats(beta=beta, acm=acm, n_grps=n_grps, alpha=alpha_i)
    stats['beta'] = beta
    return stats

## Fit models and calculate stats

In [15]:
before_behs_ref = list(set(before_behs).difference(ps['beh_ref']))
after_behs_ref = list(set(after_behs).difference(ps['beh_ref']))
before_behs_ref = sorted(before_behs_ref)
after_behs_ref = sorted(after_behs_ref)

n_before_behs = len(before_behs_ref)
n_after_behs = len(after_behs_ref)

one_hot_data_ref, one_hot_vars_ref = one_hot_from_table(data, beh_before=before_behs_ref, beh_after=after_behs_ref)
one_hot_data_ref = np.concatenate([one_hot_data_ref, np.ones([one_hot_data_ref.shape[0], 1])], axis=1)
one_hot_vars_ref = one_hot_vars_ref + ['ref']

In [16]:
n_rois = dff.shape[1]
full_stats = [stats_f(x_i=one_hot_data_ref, y_i=dff[:, r_i], g_i=g, alpha_i=ps['alpha']) for r_i in range(n_rois)]

## Package results

In [17]:
if (ps['test_type'] == 'state_dependence') or (ps['test_type'] == 'before_reporting'):
        test_behs = before_behs_ref
        pull_inds = range(0, n_before_behs)
        #test_betas = beta[0:n_before_behs]
       # test_c_ints = stats['c_ints'][:, 0:n_before_behs]
        #test_sig = stats['non_zero'][0:n_before_behs]
elif ((ps['test_type'] == 'prediction_dependence') or (ps['test_type'] == 'after_reporting') or
      (ps['test_type'] == 'decision_dependence')):
        test_behs = after_behs_ref
        pull_inds = range(n_before_behs, n_before_behs+n_after_behs)
       # test_betas = beta[n_before_behs:n_before_behs+n_after_behs]
        #test_c_ints = stats['c_ints'][:, n_before_behs:n_before_behs+n_after_behs]
       # test_sig = stats['non_zero'][n_before_behs:n_before_behs+n_after_behs]
else:
        raise(ValueError('The test_type ' + ps['test_type'] + ' is not recognized.'))
        
beh_stats = dict()
for b, p_i in zip(test_behs, pull_inds):
    beh_stats[b] = dict()
    beh_stats[b]['p_values'] = [rs_dict['non_zero_p'][p_i] for rs_dict in full_stats]
    beh_stats[b]['beta'] = [rs_dict['beta'][p_i] for rs_dict in full_stats]

## Save results

In [18]:
save_name = append_ts(ps['test_type'] + '_' + ps['save_str']) + '.pkl'
save_path = Path(ps['save_folder']) / save_name

In [19]:
rs = dict()
rs['beh_stats'] = beh_stats
rs['full_stats'] = full_stats
rs['ps'] = ps

In [20]:
with open(save_path, 'wb') as f:
    pickle.dump(rs, f)

In [21]:
print('Saved results to: ' + str(save_path))

Saved results to: \\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats\prediction_dependence_quiet_ref_A4_2020_09_26_22_03_44_324504.pkl


In [22]:
file_data

{'ps': {'data_loc_file': 'A:\\projects\\keller_vnc\\data\\experiment_data_locations.xlsx',
  'trans_file': 'A:\\projects\\keller_vnc\\data\\extracted_dff_v2\\transition_list.xlsx',
  'dataset_folder': 'extracted',
  'dataset_base_folder': 'K:\\\\SV4',
  'f_ts_str': 'f_1_5_5',
  'bl_ts_str': 'bl_1_5_5',
  'n_smps': 3,
  'background': 100,
  'ep': 20,
  'save_folder': 'A:\\projects\\keller_vnc\\results\\whole_brain_stats',
  'save_str': 'dff_1_5_5_with_ep'},
 'event_annots':          subject_id beh_before beh_after Tgt Site  Trans Time  Int Time  \
 0    CW_17-08-23-L1          F         P       A4      4.8120    0.4010   
 1    CW_17-08-23-L1          F         P       A4      4.0100    0.4010   
 2    CW_17-08-23-L2          F         B       A4      2.1804    0.3634   
 3    CW_17-08-23-L2          F         B       A4      2.1804    0.3634   
 4    CW_17-08-23-L2          F         B       A4      1.8170    0.3634   
 ..              ...        ...       ...      ...         ...     